In [1]:
import pdfplumber
import pandas as pd
import re
import os
import glob

In [37]:
os.chdir(r'D:\C++\PYTHON\PYTHON_PRACTICE\invoice\spiceinvoice')
invoices_names = glob.glob('*.pdf' )


In [12]:
def pdf_to_dictionary(pdf_name):
    # Directory of the invoices
    directory = r'D:\C++\PYTHON\PYTHON_PRACTICE\invoice\spiceinvoice'
    
    # The name of files to read, all the inovices
    file_to_load = os.path.join(directory,pdf_name)
    
    #print(file_to_load)
    # Loading the invoice
    pdf = pdfplumber.open(file_to_load)
    
    # Extracting the text from the pdf file
    text = pdf.pages[0].extract_text()
    
    # Printing the text, I'll remove this 
    #print(text)
    
    #  Exhautive list of all the columns,
    columns = ['Invoice No','Invoice Date','Passenger','PNR',
          'Sector','GSTIN / UIN of Customer','Place of Supply',
          'GSTN Registered Customer Name','State Code','Email of Registered Customer',
          'Currency','Description','SAC code','Taxable Value',
          'Non Taxable/ Exempted Value','IGST Rate %','IGSC Amount',
          'CGST Rate', 'CGST Amount','SGST/UTGST Rate','SGST/UTGST Amount',
          'Whether Reverse charge applicable (Y/N)']
    
    # Okay , so taking care of this space issue
    text_to_read =(((text.replace('\xa0','#')).replace('\xad','')).replace('#',' ')).splitlines()
    #f
    # keys to take the first part of the invoice
    
    keys = ['Invoice No','Invoice Date','Passenger','PNR','Sector','GSTIN / UIN of Customer','Place of Supply',
          'GSTN Registered Customer Name','State Code','Email of Registered Customer',
          'Currency']
    key_to_remove = ['Sector','Place of Supply','State Code','Currency']

    fields = []
    
    # MAGIC MAGIC !!
    for line in text_to_read:
        for key in keys:
            if key in line:
                #print(line)
                string = line
                #print(string.split(key,1)[1][2:])
                value = string.split(key,1)[1][2:]
                for k in key_to_remove:
                    if k in value:
                        #print(value.split(k,1)[0])
                        value = value.split(k,1)[0]
                fields.append(value)
                continue
                
    # first part of the data
    dct = dict(zip(keys, fields))
    
    #print('Length Keys: ',len(keys))
    #print('Length Fields: ',len(fileds))
    
    charges_key = '1 Transport of passenger by air'

    for line in text_to_read:
        if charges_key in line:
            charges = line.split(charges_key,1)[1]
            #print(charges)


    charges_seperated = charges.split()

    bill_charge_key = ['SAC','Taxable Value','Total Value',
                       'IGST_Rate(%)','IGST_Amount',
                       'CGST_Rate(%)','CGST_Amount',
                       'SGST_Rate(%)','SGST_Amount']


    amount= dict.fromkeys(bill_charge_key, None) 

    # Filling the values
    for l in range(0,3):
        amount[bill_charge_key[l]] = charges_seperated[l]

    tax = ['IGST_Rate(%)','IGST_Amount',
            'CGST_Rate(%)','CGST_Amount',
            'SGST_Rate(%)','SGST_Amount']   
    # Filling the taxes

    #print(charges_seperated)
    
    # MORE MAGIC MAGIC !!!
    if charges_seperated[3] == '5.00':
        amount[bill_charge_key[3]] = charges_seperated[3]
        amount[bill_charge_key[4]] = charges_seperated[5]
        amount[bill_charge_key[5]] = '0'
        amount[bill_charge_key[6]] = '0'
        amount[bill_charge_key[7]] = '0'
        amount[bill_charge_key[8]] = '0'

    else:
        amount[bill_charge_key[3]] = 0
        amount[bill_charge_key[4]] = 0
        amount[bill_charge_key[5]] = charges_seperated[3]
        amount[bill_charge_key[6]] = charges_seperated[5]
        amount[bill_charge_key[7]] = charges_seperated[3]
        amount[bill_charge_key[8]] = charges_seperated[5]

    #amount
    
    final = {**dct, **amount}
    #print(final)
    return final

In [32]:
all_invoice_dictionary = []
for i in range(0,len(invoices_names)):
    dictionary_temp = pdf_to_dictionary(invoices_names[i])
    all_invoice_dictionary.append(dictionary_temp)
    


In [38]:
spice_jet_invoice_data = pd.DataFrame(all_invoice_dictionary)

file_name = r'D:\C++\PYTHON\PYTHON_PRACTICE\invoice\spiceinvoice\spice_jet_invoice.csv'
spice_jet_invoice_data.to_csv(file_name, encoding='utf-8', header=True)